In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from bs4 import BeautifulSoup
import unicodedata
import string
import re
import nltk

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [71]:
all_letters = string.ascii_letters + " .,;'-"
df = pd.read_csv("./data/IMDB Dataset.csv")
df.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [133]:
def preprocess(text):
    cleanr = re.compile('<.*?>')
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = re.sub(cleanr, ' ', text)
    text = text.lower()
    text = regex.sub(' ', text)
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
    return text
def word_process(text):
    text = text.split(" ")
    text = [word for word in text if word not in stopwords]
    return text

def encoding(text):
    text = text.split(" ")
    text = [word2index[word] for word in text]
    return text
    

In [128]:
text['review'] = df['review'].apply(preprocess)
text.head(2)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming t...,positive


In [129]:
all_words = word_process(" ".join(list(text['review'])))
words = list(set(all_words))
print(len(words))

99877


In [140]:
word2index = {}
index2word = {}
for i,word in enumerate(words):
    index2word[i] = word
    word2index[word] = i

In [141]:
encoding(" ".join(word_process(preprocess(df['review'][0]))))

[30482,
 7466,
 67263,
 8532,
 0,
 93487,
 42939,
 69621,
 0,
 85329,
 0,
 72411,
 55666,
 0,
 0,
 94012,
 74387,
 10869,
 93487,
 91285,
 36907,
 61570,
 29167,
 0,
 64685,
 85329,
 19231,
 34575,
 0,
 27740,
 0,
 96170,
 45917,
 92200,
 23292,
 0,
 96170,
 26675,
 85395,
 68598,
 78449,
 0,
 87408,
 29167,
 0,
 91181,
 0,
 48839,
 54272,
 19231,
 0,
 0,
 47900,
 93487,
 49320,
 20035,
 95388,
 71730,
 65258,
 92398,
 69748,
 0,
 32613,
 7330,
 66830,
 30510,
 0,
 25182,
 93144,
 30317,
 47186,
 56725,
 17848,
 12636,
 78327,
 0,
 11595,
 56471,
 6277,
 0,
 81356,
 30510,
 39839,
 58703,
 0,
 6564,
 0,
 29084,
 0,
 69313,
 0,
 35011,
 0,
 4203,
 0,
 92669,
 0,
 83956,
 0,
 0,
 0,
 4530,
 0,
 60911,
 28920,
 0,
 63592,
 63210,
 767,
 76942,
 26773,
 55357,
 73584,
 0,
 0,
 32471,
 24801,
 93736,
 47918,
 96170,
 84710,
 74184,
 16191,
 71512,
 65165,
 0,
 76320,
 30620,
 39277,
 90879,
 74620,
 94064,
 0,
 76320,
 62709,
 0,
 76320,
 1870,
 0,
 0,
 93487,
 58876,
 46156,
 0,
 94012,
 4

In [ ]:
word2index['accuses']